# TABLE OF CONTENTS

* Part one

* Part Two [Calculating PECDS](#PART_TWO)

In [ ]:
#import potential packages needed

In [ ]:
#system packages

import sys
import warnings
import os 
import traceback #obs? 
if not sys.warnoptions:
    warnings.filterwarnings("once")  

In [ ]:
#base packages:

import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats, integrate, optimize
import math
import datetime
from datetime import date, timedelta

In [ ]:
#optional packages

from statsmodels.tsa.base.datetools import dates_from_str
from IPython.display import display
pd.options.display.max_columns = 50  #allow DF.head to show all columns in notebook
from see import see
from tabulate import tabulate 

In [ ]:
#packages for the econometrics / models

from statsmodels.tsa.vector_ar import vecm
from statsmodels.tsa.stattools import adfuller
from arch.unitroot import DFGLS, ADF, KPSS, PhillipsPerron, ZivotAndrews
from arch.unitroot.cointegration import engle_granger, phillips_ouliaris
import statsmodels.formula.api as smf  #VAR package contained within 
import statsmodels.tsa.api as smt

#import the functionality for detecting mathematical errors (E.G. types of linear algebra issues etc.)
from statsmodels.tools.sm_exceptions import ValueWarning
from arch.utility.exceptions import (
    InfeasibleTestException,
    InvalidLengthWarning,
    invalid_length_doc)
warnings.filterwarnings("once", category = ValueWarning)

## Table of Contents

1) Data Overview & Data Cleaning Process

# Section 1.1) Data Overview

1) First data point is a .txt file (~10GB) of Bond Data, downloaded from Wharton Research/Data Services (WRDS): pertaining to TRACE bond trades for all regular bonds (senior bonds with fixed or zero coupons) issued by firms in Compustat. Observations are at the day-level
    * contains all bond trades between 2002 and June-2017
    * Data included is bond ID, cusip_id, exact trade date/time, traded price, quoted_yield
    
2) Second data point is Bond Maturities information (From TRACE) .txt file from WRDS, relating to the corresponding data on bond maturities seniority etc.
    * contains bond ID, cusip_id, sub-product type, debt-type, issuer_name, maturity date, grade, convertible_flag, company_symbol
    
3) Third data point is Bond Coupon Rates,  a .txt file from (Mergent FISD), pertaining to coupon information per bond
    * contains: issue_id, coupon_type, offering_date, principal, first_interest_date, interest_frequency, coupon, day_count_basis, last_interest_date
    
4) Fourth data entry is information extracted from Fred, relating to information on swap_rates from one-year through to five-year (the risk-free rate proxy in this report)- all denonted in USD; in addition to values of VIX and other interest rate components

5) Fifth data entry is the trace-mergent linkfile (provides a common factor to merge bond coupon rae information with bond market prices and bond maturities)
    * contains: bond_ID, issue_id, issue_id_fisd
    * This file is necessary to merge together the data about the bonds as WRDS has data on coupons, maturities, prices from different sub-vendors who potentially use different codes/reference numbers. This file will enable us to merge all of the above information on bonds and eventually, with their respective Credit Default Swap (CDS) counterparts. 
    
6) Sixth date entry is CDS spreads (From Datastream) .txt file (>20GB) of CDS trades on the same date range as the earlier bond information. Only CDS with standard contractual clauses are considered. Observations are at the day-level
    * contains information on: series_id, gvkey, company_name, stock_ticker, source, duration, clause, currency, class_type, date, cds_spread
    
7) (By earlier filtered viable GVKEY sub-sample) Equivalent Equity trading data (Daily) from WRDS/Compustat (CSV ~1GB)
    * Includes: open, high, low, close, trading_volume......

8) (By earlier filtered viable GVKEY sub-sample)(Data is monthly/quarterly level) of: Firm-level ratios
    * pertaining to: EPS....
    
9) Equity Sector Data & Returns (Daily-level) from WRDS/Compustat     

10) (By earlier filtered viable GVKEY sub-sample) detailed breakdown of opening, high, low, closing bond-trades data


#### SIDENOTE: Additional robustness checks will be performed on the same data but re-sampled by weekly/monthly - these DFs will be based off the final DF which includes the merging of all the above dataframes (#1 through #10) but re-sampled respectively.

11) weekly_resample_df

12) monthly_resample_df
    

In [ ]:
#the try/except needs to be changed to the more recent format - with location etc. 


#pecds_1.30 version below 

In [ ]:
#bonds: Trade Day(date), bond_sym_id, price 
bond_file = open('sample_bond_data_all.txt','r')
bonds_date_price = {}

for line in bond_file:  
  line = line.rstrip("\r\n") 
  [bond_sym_id, date, trade_time, quantity, price, _yield] = line.split("\t")
  if bond_sym_id=="bond_sym_id" and date=='date':
    continue
  try:
      bonds_date_price[(date, bond_sym_id)] = price
  except ValueError:
    pass

try:
    bonds_date_price_df = pd.DataFrame(list(bonds_date_price.items()), columns=['date','date_id_price'])
except Exception:
    trackback.print_exc()
     
bond_file.close()

In [ ]:
#Create a datetime object for later merging this data with the relevant swap_rates / other data
format_date = '%Y%m%d'                                                                #American style date
datetime_obj = bonds_date_price_df['trade_date'].apply(lambda x: datetime.datetime.strptime(x, format_date))

#set the new datetime as DF index
bonds_date_price_df = bonds_date_price_df.set_index('trade_date')

In [ ]:
bond_file= open('master_file.txt','r')
bond_maturities = {}

for line in bond_file:  
  line = line.rstrip("\r\n") 
  [bond_sym_id, cusip_id, bsym_id, sub_prdct_type, debt_type_cd, issuer_nm, scrty_ds, cpn_rt, cpn_type_cd, trd_rpt_efctv_dt, mtrty_dt, grade, ind_144a, dissem, cnvrb_fl, company_symbol] = line.split("\t")
  if bond_sym_id=="bond_sym_id":
    continue
  try:
      bond_maturities[(bond_sym_id)] = mtrty_dt
  except ValueError:
    pass

try:
    bond_maturities_df = pd.DataFrame(list(bond_maturities.items()), columns=['bond_sym_id','maturity_date'])
except Exception:
    trackback.print_exc()
     
bond_file.close()

In [ ]:
#matched with the first file by bond_ID

In [ ]:
coupon_file= open('coupon_info.txt','r')
coupons_dict = {}

#change to include the location in the try/except loop

for line in coupon_file:  
  line = line.rstrip("\r\n")
  [ISSUE_ID, MATURITY, COUPON_TYPE, OFFERING_DATE, PRINCIPAL_AMT, FIRST_INTEREST_DATE,
   INTEREST_FREQUENCY, COUPON, DAY_COUNT_BASIS, LAST_INTEREST_DATE] = line.split("\t")
  if ISSUE_ID=="ISSUE_ID":
    continue
  try:
      coupons_dict[(ISSUE_ID, INTEREST_FREQUENCY, COUPON)] = ISSUE_ID, INTEREST_FREQUENCY, COUPON
  except ValueError:
    pass

try:
    coupons_df = pd.DataFrame(list(coupons_dict.items()), columns=['ISSUE_ID','INTEREST_FREQ'])
except RuntimeError as re:
    print("runtime error", re)
except Exception as other:
    print("something else", other)
 
coupon_file.close()

In [ ]:
#RFR DATA

In [ ]:
from fredapi import Fred
fred = Fred(api_key='56e2cc23702c09f0c02226f2780c4de4') #censor this code eventually

In [ ]:
#SWAP RATES



get from older version 


In [ ]:
#LINEAR INTERPOLATION OF THE RFRs

#interpolate / line-broken for readability
fulldata_df[['riskfree_rate_1yr', 'riskfree_rate_2yr', 'riskfree_rate_3yr', 'riskfree_rate_4yr', 'riskfree_rate_5yr']] = 
fulldata_df[['riskfree_rate_1yr', 'riskfree_rate_2yr', 'riskfree_rate_3yr', 'riskfree_rate_4yr', 'riskfree_rate_5yr']].interpolate(method='linear',limit_direction='forward')

#method = linear; consecutive NaN(s) are filled forward; axis = 1, (interpolation occurs down the columns )

In [ ]:
#VIX / TED SPREAD - also from FRED

#VIX series
cboe_vix = fred.get_series('VIXCLS', '2006-07-03','2017-06-30')

#TEDRATE
ted_rate = fred.get_series('TEDRATE', '2006-07-03','2017-06-30')

#concatenate the two series column-wise
int_spreads_concat = pd.concat([cboe_vix, ted_rate],axis=1)

#convert to a dataframe
interest_spreads_df = pd.DataFrame(int_spreads_concat.values, 
                                   index=cboe_vix.index,
                                  columns=['cboe_vix','ted_rate'])

#interpolate the DF (linearly) any missing values in the: VIX or TED rates
interest_spreads_df[['cboe_vix','ted_rate']] = interest_spreads_df[['cboe_vix','ted_rate']].interpolate(method='linear',limit_direction='forward')

In [ ]:
#check both series have zero NaN(s)
interest_spreads_df.isna().sum()

In [ ]:
#reset the index of the DF so that the trade-date becomes a seperate column (to allow merging onto the main bond/cds DF)
interest_spreads_df = interest_spreads_df.reset_index()

#rename the relevant columns:
interest_spreads_df = interest_spreads_df.rename(columns={'index':'trade_date'})

In [ ]:
#check d-types (datetime64 for trade/date and float64 for values)
print(interest_spreads_df.dtypes)

### Merge this DF onto the main dateframe with the firm-level data
* set the main-DF as the principal DF, such that a left merge on trade-date will only keep the relevant dates for each firm
* that is, E.G. if Firm(X) has data from 2010:2014, only the interest_spreads from 2010:2014 will be affixed for this firm 

In [ ]:
#E.G. of the merge: 
fulldata_daily_vix_df = fulldata_daily_df.merge(interest_spreads_df, how='left', on='trade_date')

In [ ]:
#trace_mergent_linkfile.txt
merge_file= open('trace_mergent_linkfile.txt','r')
merge_dict = {}

for line in merge_file:  
  line = line.rstrip("\r\n")
  [bond_sym_id, issue_id_fisd] = line.split("\t")
  if bond_sym_id=="bond_sym_id":
    continue
  try:
      merge_dict[bond_sym_id, issue_id_fisd] = bond_sym_id, issue_id_fisd
  except ValueError:
    pass

try:
    mergefile_df = pd.DataFrame(list(merge_dict.items()), columns=['bond_sym_id','issue_id_fisd'])
except (RuntimeError, TypeError, NameError):
    print('ERROR')
     
merge_file.close()

In [ ]:
#CDS SPREAD

cds_file = open('sample_cds_data_all.txt','r')
cds_spread_dict = {}

for line in cds_file:  
  line = line.rstrip("\r\n") 
  [series_id, series_name, gvkey, company_name, ticker, source, duration, clause, currency, class_type, date, cds_spread] = line.split("\t")   
  if series_id=="series_id":
    continue
  if (float(duration)==5 and currency=="USD" and clause=="XR"):
    cds_spread_dict[gvkey, series_id, date] = cds_spread #date removed 
    
cds_file.close()

try: 
    cds_spread_df2 = pd.DataFrame(list(cds_spread_dict.items()),columns = ['cds_info','cds_spread']) 
except RuntimeError as re:
    print("runtime error", re)
except Exception as other:
    print("something else", other)

# DATAFRAME (#8) - FIRM LEVEL RATIOS
#### E.G. of ratios contained within & outlining what the Wharton abreviations actually represent: 

* #bm = book/market
* #pe_inc = P/E (Diluted, Incl. EI)
* #ps = price/sales
* #p/cf = Price/Cash flow	
* #npm = net profit margin
* #roe = return on equity 
* #Gprof = Gross profit / Total Assets
* #Capital_ratio = Capitalization ratio
* #fcf_ocf = Free Cash Flow/Operating Cash Flow
* #de_ratio = Total Debt/Equity
* #curr_ratio = current ratio 
* #rd_sale = Research and Development/Sales
* #ptb = price to book
* #divyield = dividend yield 

#### NOTE: due to the inherently different nature of the included firms, it is expected that there will be (Null) values in each ratio per firm, E.G. not all firms will pay a dividend and thus dividend_yield will be 0.

In [ ]:
firm_level_ratios = 'firm_level_ratios_gvkey.csv'
firm_level_ratio_df = pd.read_csv(firm_level_ratios)

In [ ]:
#because of the .CSV file, the dtypes won't be compatible with the existing pandas-based DF
#principally, the date series (for merging onto the main DF) will need to be converted to datetime from string

#convert the float64 column to datetime
#in Wharton: the series for which the ratio was released as public information is referred to as 'public_date'
firm_level_ratios_dtime = pd.to_datetime(firm_level_ratio_df['public_date'], format='%Y%m%d', errors='coerce')

#rename this column to match 'trade_date' so it can be merged 
firm_level_ratios_dtime = firm_level_ratios_dtime.rename(columns={"public_date": "trade_date"})

## Overcoming a major issue: different frequencies of reporting, as ratios per firm are reported either monthly, or quarterly:
* Need to devise a method to interpolate the values between these two points in time and transform it to daily-level values so some series can later be used in the various models, E.G. Vector Autoregressions 
* Also, need to make sure the method cuts the time periods correctly for each firm-combination, as virtually every firm-combination has a unique set of start and finish dates for its CDS/BOND data series: both for accuracy and to reduce computation strain 

In [ ]:
#also noted that we will need to merge by [GVKEY, DATE] now and thus GVKEY needs to be converted from (object) type
firm_level_ratios_dtime['gvkey'] = firm_level_ratios_dtime['gvkey'].astype(np.int)

In [ ]:
#create a seperate date column which is just the month:year and leave the daily-level series unaffected
firm_level_ratios_dtime['trade_date_month'] = firm_level_ratios_dtime['trade_date']

#convert to date-time, the level of the ratios data is at minimum (month:year)
firm_level_ratios_dtime['trade_date_month'] = pd.to_datetime(firm_level_ratios_dtime['trade_date_month'], format='%Y-%m')

#create a pivot-tabel of those value-series deemed relevant for the later regression models
df_firm_ratios_mini = firm_level_ratios_dtime.pivot(index='trade_date_month', columns='gvkey',
                                                     values=['ps','pcf','GProf','capital_ratio','de_ratio','rd_sale'])

#now, create the offsets for the DF's dates: 
start_date = df_firm_ratios_mini.index.min() - pd.DateOffset(day=1)    #note: "trade_date_month" is now the index column
end_date = df_firm_ratios_mini.index.max() + pd.DateOffset(day=31)
dates = pd.date_range(start_date, end_date, freq='D')
dates.name = 'date'

#ffill will pad the values in-between the two dates to create a continuous flow E.G. January 1, 2,... 31st etc.
pivot_df_firm_ratios_mini = df_firm_ratios_mini.reindex(dates, method='ffill') 

In [ ]:
#now, need to rearrange the pivot-DF back to a regular format by stacking the DF on the GVKEY
pivot_df_firm_ratios_mini = pivot_df_firm_ratios_mini.stack('gvkey')

#sort the DF, so that each GVKEY is sorted chronologically
pivot_df_firm_ratios_mini = pivot_df_firm_ratios_mini.sort_index()

#reset the index, so that this chronologically sorted date-series is now treated as a column within the DF (for easier merging)
pivot_df_firm_ratios_mini = pivot_df_firm_ratios_mini.reset_index()

In [ ]:
#Optional: create a pickle of this particular set of variables, 
#if it is required to change the input-series for a different set of values

pivot_df_firm_ratios_mini.to_pickle("./pivot_df_firm_ratios_mini.pkl")
#pivot_df_firm_ratios_mini = pd.read_pickle("./pivot_df_firm_ratios_mini.pkl")  #would be read-pickle code

In [ ]:
#now, the final merge to re-affix the now converted/padded values of firm-level ratios on a daily level to the main DF
#optional to rename DF to modified version, e.g. fulldata_df_ratios to confirm the ratios are correctly formatted now

fulldata_df = fulldata_df.merge(pivot_df_firm_ratios_mini, how='left', on=['gvkey','trade_date'])

# DATAFRAME (#9) DAILY EQUITY PRICES / TRADING INDICATORS

In [ ]:
#DVRATED -- Indicated Annual Dividend Rate - Daily
#curcdd = CURCDD -- ISO Currency Code - Daily
#cshoc = CSHOC -- Shares Outstanding
#cshtrd = CSHTRD -- Trading Volume - Daily
#eps = EPS -- Current EPS
#prccd = PRCCD -- Price - Close - Daily
#prchd = PRCHD -- Price - High - Daily
#prcld = PRCLD -- Price - Low - Daily
#prcod = PRCOD -- Price - Open - Daily
#exchg = Stock Exchange Code	 #19 = other-OTC US, 12 = Amaerican, 14 = NASDAQ, 11 = NYSE
#secstat = Security Status Market	 I = inactive, A= active
#costat = COSTAT -- Active/Inactive Status Marker

# = GSUBIND -- GIC Sub-Industries
#Global Industry Classification Standard (GICS). The Sub-industry is represented by all 8 digits of the total GICS code.

#idbflag = IDBFLAG -- International, Domestic, Both Indicator (B = North-America & Int, D = domestic (NA), I = int)
#spicindcd = SPCINDCD -- S&P Industry Sector Code

In [ ]:
daily_us_equity_prices = 'gvkey_daily_usa_equity_prices.csv'
daily_us_equity_prices_df = pd.read_csv(daily_us_equity_prices)

### Given this data is already at the daily-level by GVKEY, the process to merge it onto the main-DF is virtually identical to the processes outlined above. Hence, it has been omitted here to avoid needless repitition 

In [ ]:
#the one thing to note here is to ensure that you keep only the one trading observation for prices per day:
#E.G. after converting the date to datetime, renaming columns, checking for Null/NaN etc.

daily_us_equity_prices_df = daily_us_equity_prices_df.drop_duplicates(subset=['gvkey','trade_date'], keep='first')

# Breakingdown the data merging process:

* I dropped all bonds with missing or negative prices (assume this is an error in the data on the vendor's end as it is implausible to have a negatively priced bond); repeated this step with other important data points, such as: missing coupon rates or maturity dates. 
    * Out of 10M + total observations, only a few thousand were dropped in this manner so the data integrity as whole is sound. 
    * Around 10% of the bonds have no data on interest_frequency (frequency of the coupon payments, usually semi-annual for most fixed-rate bonds); this issue is elaborated on more in one of the next steps


* Multiple approaches in merging, I believe a fairly simple method is to perform an outer-join on the BOND & CDS data frames; thus, potentially there will be some observations with bond prices and no CDS spreads, and vice-versa. However, quickly dropping the NaN from the DF will resolve this, resulting in a DF with just the overlapping firms with both CDS and BOND data. 



In [ ]:
#th

## Restricting the aggregate DF to only those firms with both a CDS:Bond trading between 4-6 years
* First, calculate the time gap between date traded (From the original bond trade file) and the maturity date by bond_ID (from the 2nd data entry file on bond maturities)- to filter the DF to only those observatios beteen four and six years

In [ ]:
#Convert the maturity date column to more workable datetime format for the later date(1) - date(2) calculation 
fulldata_df['maturity_date'] = pd.to_datetime(fulldata_df['maturity_date'], format="%Y%m%d", errors='coerce')

#Extract the existing dates within the index as seperate column to simplify the subsequent calculation
fulldata_df['time_dif'] = ((fulldata_df['maturity_date'] - fulldata_df['trade_date']))

In [ ]:
#filter sample to keep only observations for which both a CDS-Bond combination exist with maturities between 4-6 years

#pre-create the variables to hold the date limits (note, 1Y == 365.2425 days)
fouryear_timedelta = timedelta(days=(365.2425 * 4))
sixyear_timedelta = timedelta(days=(365.2425 * 6))

#first, restrict to only maturities less than 6 years
fulldata_df = fulldata_df[fulldata_df['time_dif'] < pd.Timedelta(sixyear_timedelta, unit='d')]

#further filter to only those maturities greater than 4 years; resulting in a range of 4-6Y
fulldata_df = fulldata_df[fulldata_df['time_dif'] > pd.Timedelta(fouryear_timedelta, unit='d')]


In [ ]:
#improve the readability of the dataframe above by converting the days to years
#fill NaN or missing values with 0 days

fulldata_df['time_dif'] = (fulldata_df['time_dif'] / np.timedelta64(1, 'Y')).fillna(pd.Timedelta('0 Days'))

#optional but recommended: round the large float value for a more compact, readable DF
fulldata_df['time_dif'] = np.round(fulldata_df['time_dif'], 3)

# Basic Data Cleaning & Preparation

* Checking for NaN / Missing / Duplicated Values

In [ ]:
#after merging all of the previous data into one aggregate frame
#note: pickles of the intermediate data sets (E.G. just equities / bonds / cds, without ratios) exist for faster computation
#although, pandas can handle a 1.34m row dataframe without much effort so this is largely unnecessary

fulldata_df = pd.read_table('full_dataset.txt')  
# ROWS = 13445375 

fulldata_df = fulldata_df[fulldata_df['bond_sym_id'].notna()] #remove the NaN in the bond_sym_id series
# total rows from (13445375) to 11202753

fulldata_df = fulldata_df[fulldata_df['cds_spread'].notna()] #drop NaN from CDS_spread
# total rows from 11202753 to 5495942

### Inspecting a DF of the NaN / Missing values revealed that there were a few corrupted data points with {} in place of the data
* There were 31 instances of corrupted data points
* Also noted several thousand NaN in the (bond) interest/coupon frequency; however, these require seperate considerations

In [ ]:
#find all the occurences of the corrupted data points with: {} instead of price (#NO = 31)
corrupted_price_df = fulldata_df[fulldata_df.price.str.contains('\{}')==True]

#send the index numbers to a list
corrupted_indexes = corrupted_price_df.index.tolist()

#find the loc of these indexs so faciliate dropping them from the DF 
corrupted_indexloc = []
for i in corrupted_indexes:
    corrupted_indexloc.append(fulldata_df.index.get_loc(i))
    

In [ ]:
#drop all of the rows with the corrupted {} in place of the price for the entire DF: based off their index.loc list above
fulldata_df = fulldata_df.drop(fulldata_df.index[corrupted_indexloc], axis=0)

### A feature of the data is missing interest_frequency data points; these cannot simply be dropped as they potentially represent either: zero-coupon bonds, bonds which for whatever reason (perhaps near bankruptcy and undergoing debt deorganisation) have temporarily ceased paying coupons
* These points must be accounted for in the subsequent code to calculate their par-equivalent CDS spreads & how this might affect the results as a whole

In [ ]:
#inspect coupon_freq NaN(s)
missing_ir_freq = fulldata_df[fulldata_df['interest_frequency'].isna()]
missing_ir_freq

#revealed 47219 instances of missing coupon frequency data

# Conclusion on the data cleaning process: 


* The result is roughly 408 firms with outstanding CDS contrats, bonds which mature in 4-6 years, as well as have information available on the bonds coupons 

<a id='PART_TWO'></a>

# PART TWO

### Calculating Par-Equivalent Credit Default Swap (PECDS) 